In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import log_loss


In [2]:
df = pd.read_csv("./task_2022/train_data_swc.csv")

Light exploration

In [3]:
df.head()

,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,...,x_100,x_101,x_102,x_103,x_104,x_105,x_106,x_107,x_108,y
0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.2,0.8,0.0,0.0,0.0,0.0,0.6,0.2,0.0,0.0,...,0.2,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,9
2,0.0,0.0,0.0,0.4,0.0,0.0,3.2,0.0,0.0,0.0,...,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,0.0,0.0,0.4,0.0,0.0,0.0,0.4,0.0,0.0,0.6,...,0.0,0.0,0.0,0.6,0.2,0.0,0.0,0.2,0.0,9


In [4]:
target = "y"

In [5]:
unique_classes = df[target].nunique()
print(f"Number of unique classes in {target}: {unique_classes}")

Number of unique classes in y: 9


# Feature Creation

In [6]:
# Config
top_n_features = 30 # 2n+n*(n-1) features at the final

In [8]:
from lightgbm import LGBMClassifier
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Fit a LGBM model to find important features
lgbm = LGBMClassifier(objective='multiclass', num_class=9)
lgbm.fit(X_train, y_train)

# Get feature importances
importances = lgbm.feature_importances_

# Get the top 10 most important feature names
top_10_features = X_train.columns[importances.argsort()[-top_n_features:]]

# Use only the top 10 important features for the new feature creation
X_train_imp = X_train[top_10_features].copy()
X_test_imp = X_test[top_10_features].copy()

# Create the new features on both train and test sets
for X in [X_train_imp, X_test_imp]:
    # Interaction terms
    for col1, col2 in combinations(X.columns, 2):
        X[col1+'_times_'+col2] = X[col1] * X[col2]

    # Binning
    discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
    if X is X_train_imp:
        X_binned = discretizer.fit_transform(X)
    else:
        X_binned = discretizer.transform(X)
    X_binned = pd.DataFrame(X_binned, columns=[col + '_binned' for col in X.columns])
    X = pd.concat([X, X_binned], axis=1)

    # Ratios
    for col1, col2 in combinations(X.columns, 2):
        X[col1+'_to_'+col2] = X[col1] / X[col2]


/tmp/ipykernel_30149/3697776982.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col1+'_to_'+col2] = df[col1] / df[col2]
/tmp/ipykernel_30149/3697776982.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col1+'_to_'+col2] = df[col1] / df[col2]
/tmp/ipykernel_30149/3697776982.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fr